We are given a $n \times n$ doubly stochastic matrix $P_{DS}$. By Bikrhoff's theorem, *every doubly stochastic matrix is a convex combination of permutation matrices*. Hence, let us try to find such a convex combination.

The idea is that this decomposition can give us more insights into which permutation matrices are "closest" to this doubly stochastic matrix $P_{DS}$. However, many questions arise:
- Is there a way to do this efficiently?
- Is the decomposition unique?
- Does the decomposition tell us something meaningful?

In [1]:
import numpy as np # easier matrix computations
from itertools import permutations # get all permutations
from scipy import optimize

In [2]:
n = 3
P_DS = np.array([[0.5, 0.3, 0.2], [0.2, 0.5, 0.3], [0.3, 0.2, 0.5]])
print(P_DS)

[[0.5 0.3 0.2]
 [0.2 0.5 0.3]
 [0.3 0.2 0.5]]


#### Decomposition algorithm
From [this website.](https://math.stackexchange.com/questions/214948/whats-the-algorithm-of-finding-the-convex-combination-of-permutation-matrices-f)

1. Iterate through all permutation matrices $P_\pi$.
    a. If $P_\pi$ and $P_{DS}$ share non-zero entries.
        i. Assign as much as possible from $P_{DS}$ TO $P_\pi$
        ii. Subtract the assigned part from $P_{DS}$.

In [57]:
def find_convex_comb(P_DS, total = -1): 
        
    # get permutation matrices 
    perms = np.array(list(permutations(np.identity(n))))
    
    # shuffle them to check if there is only one unique solution
    np.random.shuffle(perms)

    # copy to double check later
    P_DS_to_modify = P_DS.copy()

    # save our results
    results = []
    
    # if we do not want to try all factorials exhaustively
    if total == -1:
        total = np.math.factorial(n)
        
    # iterate through all permutation matrices P
    for perm in perms[:total]:  
        # find maximum value to assign to this permutation matrix
        max_assign = min(np.multiply(P_DS_to_modify, perm)[perm != 0])
    
        # subtract this part from P_DS
        P_DS_to_modify -= perm * max_assign
    
        # save results
        results.append([perm, max_assign])

    print("Sum", np.sum(results, axis = 0)[1])
    # check solution
    check = np.zeros((n, n))

    for one_iter in results:
        # retrieve results and add them to zero matrix
        perm, max_assign = one_iter[0], one_iter[1]
        check += perm * max_assign
        
        # print convex combination
        if max_assign > 0:
            print(f"Constant:{max_assign}, \n{perm}\n")

    print(f"P_DS:\n{P_DS}\n")
    print(f"Double check:\n{check}")
    # assert, if nothing shows up, it is good!
    assert check.all() == P_DS.all(), "decomposition not perfect"

n = 3
find_convex_comb(P_DS)

Sum 1.0
Constant:0.5, 
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

Constant:0.2, 
[[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]

Constant:0.3, 
[[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]

P_DS:
[[0.5 0.3 0.2]
 [0.2 0.5 0.3]
 [0.3 0.2 0.5]]

Double check:
[[0.5 0.3 0.2]
 [0.2 0.5 0.3]
 [0.3 0.2 0.5]]


In [38]:
def gen_P(n):
    P = np.random.random((n, n))
    
    rsum = 0
    csum = 0

    while (np.any(np.round(rsum, 3) != 1)) | (np.any(np.round(csum, 3) != 1)):
        P /= P.sum(0)
        P = P / P.sum(1)[:, np.newaxis]
        rsum = P.sum(1)
        csum = P.sum(0)
        
    return P

In [43]:
n = 6
P_DS_2 = gen_P(n)
find_convex_comb(P_DS_2, total = 720)

Sum 0.999695558155451
Constant:0.0641303794037869, 
[[1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]

Constant:0.010249986720648053, 
[[0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]]

Constant:0.10638867484918718, 
[[0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0.]]

Constant:0.10486812525153857, 
[[0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]]

Constant:0.10117957179810634, 
[[1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]

Constant:0.01712413869390536, 
[[0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]]

Constant:0.0355145210396

### Is the decomposition meaningful?
Now, what does this convex combination mean? If we sort the permutation matrices by size of the constant in front of it, do we get the most likely permutation matrices? Perhaps. Take for example the promising $n \times n$-dimensional matrix

$$P_{DS} = \begin{pmatrix} 1 & 0 & 0 & \cdots & 0 & 0 & 0\\ 0 & 1 & 0 & \cdots & 0 & 0 & 0 \\ 0 & 0 & 1 & \cdots&  0 & 0 & 0\\ 
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots \\  0 & 0 & 0 & \cdots & 1 & 0 & 0\\ 0 & 0 & 0 & \cdots & 0 & 0.5 & 0.5 \\ 0 & 0 & 0 & \cdots & 0 & 0.5 & 0.5 \end{pmatrix}.$$

So, the upper-left $(n-2) \times (n - 2)$ block is the identity, and only the bottom-right $2 \times 2$ block has $0.5$ as values on all entries. Now, it is clear that $I$ and $\tilde{I}$, we $\tilde{I}$ is the identity matrix, but the bottom two rows swapped, are both equally valid candidates. And indeed, if these are the first two we encounter when we are iterating over the permutation matrices, both will get a value of $0.5$ and this will be the output. Therefore, we get exactly what we want, both are equally likely and all other permutations make much lesser sense compared to these.

In [58]:
# set n to any dimension
n = 5

# create matrix above
P_DS_3 = np.identity(n)
P_DS_3[n-2:,n-2:] = [0.5, 0.5]

# find convex combo
find_convex_comb(P_DS_3)

Sum 1.0
Constant:0.5, 
[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]

Constant:0.5, 
[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]]

P_DS:
[[1.  0.  0.  0.  0. ]
 [0.  1.  0.  0.  0. ]
 [0.  0.  1.  0.  0. ]
 [0.  0.  0.  0.5 0.5]
 [0.  0.  0.  0.5 0.5]]

Double check:
[[1.  0.  0.  0.  0. ]
 [0.  1.  0.  0.  0. ]
 [0.  0.  1.  0.  0. ]
 [0.  0.  0.  0.5 0.5]
 [0.  0.  0.  0.5 0.5]]


In [50]:
# set n to any dimension
n = 10

# create matrix above
P_DS_3 = np.identity(n)
P_DS_3[n-4:,n-4:] = [0.25, 0.25, 0.25, 0.25]

# find convex combo
find_convex_comb_faster(P_DS_3)

Constant:0.25, 
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]

Constant:0.25, 
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]

Constant:0.25, 
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0

# Conclusions
There is an easy way to write a doubly stochastic matrix as a convex combination of permutation matrices. This does, however, take $\mathcal{O}(n!)$ time, which is as expected as there are $\mathcal{O}(n!)$ permutation matrices. One iteration with minimal computations suffices for every permutation matrix. In practice, I think you do not need to iterate over all permutation matrices. I noticed that e.g. half is enough to have assigneed >99% of the doucly stochastic matrix, but the running time might still be exponential.

Unfortunately, which was expected, the convex combination is *not* unique. Even for the three-dimensional case, there are multiple ways of making this convex combination. When there are multiple convex combinations, the constants may not be very meaningful.

## Faster Decomposition algorithm
While $P_{DS}$ is not the zero matrix:
- Get the most fitting permutation matrix by the hungarian algorithm.
- Use that one to subtract.


In [52]:
def find_convex_comb_faster(P_DS): 
    
    # copy to double check later
    P_DS_to_modify = P_DS.copy()

    # save our results
    results = []

    # while we have not decomposed it all
    while np.round(P_DS_to_modify, 3).any() != 0:
        # get best fitting one
        perm = closest_perm(P_DS_to_modify)
        
        # find maximum value to assign to this permutation matrix
        max_assign = min(np.multiply(P_DS_to_modify, perm)[perm != 0])
    
        # subtract this part from P_DS
        P_DS_to_modify -= perm * max_assign
    
        # print(np.round(P_DS_to_modify, 3))
        
        # save results
        results.append([perm, max_assign])

    print("Sum", np.sum(results, axis = 0)[1])
        
    # check solution
    check = np.zeros((n, n))

    for one_iter in results:
        # retrieve results and add them to zero matrix
        perm, max_assign = one_iter[0], one_iter[1]
        check += perm * max_assign
        
        # print convex combination
        if max_assign > 0:
            print(f"Constant:{max_assign}, \n{perm}\n")

            
    print(f"P_DS:\n{P_DS}\n")
    print(f"Double check:\n{check}")
    # assert, if nothing shows up, it is good!
    assert np.round(check, 3).all() == np.round(P_DS, 3).all(), "decomposition not perfect"

n = 3
find_convex_comb_faster(P_DS)

Sum 1.0
Constant:0.5, 
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

Constant:0.3, 
[[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]

Constant:0.2, 
[[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]

P_DS:
[[0.5 0.3 0.2]
 [0.2 0.5 0.3]
 [0.3 0.2 0.5]]

Double check:
[[0.5 0.3 0.2]
 [0.2 0.5 0.3]
 [0.3 0.2 0.5]]


In [47]:
def closest_perm(P_DS):
    # convert to get the maximum linear assignment problem
    P_MOD = np.ones((n, n)) * np.max(P_DS) - P_DS

    # apply the maximum linear assignment algorithm
    row_ind, col_ind = optimize.linear_sum_assignment(P_MOD)

    # initialize Permutation matrix to return
    P_perm = np.zeros((n, n))

    # fill in permutation matrix
    for row, col in zip(row_ind, col_ind):
        P_perm[row][col] = 1 
    
    # return permutation matrix
    return P_perm

In [55]:
n = 50
P_DS_2 = gen_P(n)
print(P_DS_2)
find_convex_comb_faster(P_DS_2)

[[0.01259235 0.01473306 0.00931322 ... 0.02354075 0.02513253 0.01158535]
 [0.02174186 0.02399154 0.02193752 ... 0.03083651 0.02861026 0.01758066]
 [0.00508135 0.00799245 0.00438776 ... 0.01259004 0.00343457 0.02105931]
 ...
 [0.01395255 0.01418039 0.00564013 ... 0.01551023 0.03413151 0.01246411]
 [0.02358778 0.03584182 0.00769356 ... 0.03544402 0.04380535 0.01663848]
 [0.01812144 0.02523501 0.02679748 ... 0.02102309 0.02805227 0.01996131]]
Sum 0.9908783333579484
Constant:0.02789753819419579, 
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

Constant:0.02953434626412703, 
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

Constant:0.028851578539460875, 
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0.

In [62]:
Pa = np.array([[1, 0, 0], [0, 0.51, 0.49], [0, 0.49, 0.51]])
Aa = np.array([[0.5, 0, 0], [0.5, 0, 0], [0.5, 0.5, 0.5]])
print(np.matmul(np.linalg.inv(Pa), np.matmul(Aa, Pa)))

[[  0.5    0.     0.  ]
 [  0.5  -12.25 -12.25]
 [  0.5   12.75  12.75]]
